# Environement Setup

In [ ]:
# @title 1. 🛠️ Installazione Ambiente (PyTorch 2.4.0)
import os
import sys

print("🔄 Preparazione ambiente in corso... (richiederà 2-3 minuti)")

# 1. Disinstallazione versioni incompatibili
os.system("pip uninstall -y torch torchvision torchaudio torch-scatter torch-sparse torch-cluster torch-geometric")

# 2. Installazione PyTorch 2.4.0 (Versione richiesta dal README)
# Usiamo CUDA 12.1 che è stabile su Colab
cmd_torch = "pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121"
os.system(cmd_torch)

# 3. Installazione librerie Grafiche (PyG) specifiche per Torch 2.4.0
pyg_url = "https://data.pyg.org/whl/torch-2.4.0+cu121.html"
libs = ["torch-scatter", "torch-sparse", "torch-cluster", "torch-spline-conv", "torch-kmeans"]
cmd_pyg = f"pip install {' '.join(libs)} -f {pyg_url}"
os.system(cmd_pyg)

# 4. Altre dipendenze
os.system("pip install torch-geometric hydra-core omegaconf submitit pandas ftfy")

os.system("pip install -r requirements.txt")

print("✅ Installazione completata.")
print("⚠️ IMPORTANTE: Ora vai nel menu in alto: 'Runtime' > 'Restart Session' (Riavvia Sessione) prima di eseguire la cella successiva!")

🔄 Preparazione ambiente in corso... (richiederà 2-3 minuti)
✅ Installazione completata.
⚠️ IMPORTANTE: Ora vai nel menu in alto: 'Runtime' > 'Restart Session' (Riavvia Sessione) prima di eseguire la cella successiva!


In [ ]:
# @title 2. 📂 Setup HiERO e Download Pesi Pre-addestrati
import os

# 1. Clone del repository
if not os.path.exists("HiERO"):
    print("⬇️ Clonazione repository HiERO...")
    os.system("git clone --recursive https://github.com/MarcoPernoVDP/HiERO-main.git")

os.chdir("/content/HiERO-main")

# 2. Creazione struttura cartelle
dirs = [
    "checkpoints",
    "pretrained",
    "data/ego4d/raw/annotations",
    "data/ego4d/raw/features/egovlp"
]
for d in dirs:
    os.makedirs(d, exist_ok=True)

# 3. Download del modello HiERO già addestrato (Backbone EgoVLP)
if not os.path.exists("checkpoints/hiero_egovlp.pth"):
    print("⬇️ Scaricamento Modello Pre-addestrato (hiero_egovlp.pth)...")
    os.system("wget -O checkpoints/hiero_egovlp.pth https://www.sapeirone.it/data/hiero/ckpt/hiero_egovlp.pth")

# 4. Download componenti testuali (richiesti dal codice)
print("⬇️ Scaricamento Encoder Testuale...")
os.system("wget -q -O pretrained/egovlp_text.pth https://www.sapeirone.it/data/hiero/pretrained/egovlp_text.pth")
os.system("wget -q -O pretrained/egovlp_txt_proj.pth https://www.sapeirone.it/data/hiero/pretrained/egovlp_txt_proj.pth")

# 5. Download Annotazioni necessarie (EgoMCQ/EgoClip)
print("⬇️ Scaricamento Annotazioni...")
os.system("gdown --id 1-aaDu_Gi-Y2sQI_2rsI2D1zvQBJnHpXl -O data/ego4d/raw/annotations/egoclip.csv")
os.system("gdown --id 1-5iRYf4BCHmj4MYQYFRMY4bhsWJUN3rW -O data/ego4d/raw/annotations/egomcq.json")

print("✅ Setup completato. Cartelle pronte.")

⬇️ Clonazione repository HiERO...
⬇️ Scaricamento Modello Pre-addestrato (hiero_egovlp.pth)...
⬇️ Scaricamento Encoder Testuale...
⬇️ Scaricamento Annotazioni...
✅ Setup completato. Cartelle pronte.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import zipfile

zip_path = "/content/drive/MyDrive/MistakeDetection/egovlp.zip"
extract_path = "/content/data/egovlp"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ ZIP estratto correttamente")

Mounted at /content/drive
✅ ZIP estratto correttamente


In [ ]:
# Prima vai nella directory del repo
%cd /content/HiERO-main

# Poi esegui lo script
!python infer.py \
  --checkpoint /content/HiERO-main/checkpoints/hiero_egovlp.pth \
  --input /content/data/egovlp/egovlp_test/ \
  --output /content/outputs/embeddings \
  --num_clusters 8 \
  --device cuda

/content/HiERO-main
2025-12-19 23:23:12.501403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766186592.518652    9117 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766186592.523696    9117 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766186592.536935    9117 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766186592.536963    9117 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766186592.536972    9117 computation_placer.cc:177] co